In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import cross_validation,metrics
from sklearn.grid_search import GridSearchCV
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12,4

In [2]:
def modelfit(alg,dtrain,dtest,predictiors, performCV = True, printFeatureImportance=True, cv_folds = 5):
    alg.fit(dtrain[predictors],dtrain['Is_Shortlisted'])
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    if performCV:
        cv_score = cross_validation.cross_val_score(alg,dtrain[predictors],dtrain['Is_Shortlisted'], cv = cv_folds, scoring = 'roc_auc')
        print "CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g"%(np.mean(cv_score),np.std(cv_score),np.min(cv_score),
                                                                        np.max(cv_score))
    print "Accuracy - %.4g" % metrics.accuracy_score(dtrain['Is_Shortlisted'],dtrain_predictions)
    print "AUC Score - %f" % metrics.roc_auc_score(dtrain['Is_Shortlisted'],dtrain_predprob)
    
    if printFeatureImportance:
        feat_imp = pd.Series(alg.feature_importances_,predictors).sort_values(ascending=False)
        feat_imp.plot(kind = 'bar', title = 'Feature Importance')
        plt.ylabel('Feature Importance Score')


In [3]:
data = pd.read_csv("C:/Users/user/Documents/data5.csv")
internship = pd.read_csv("C:/Users/user/Documents/Internship5.csv")
student = pd.read_csv("C:/Users/user/Documents/Student5.csv")
student_exp = pd.read_csv("C:/Users/user/Documents/Student_Exp5.csv")
data.drop('Unnamed: 0',axis=1,inplace = True)
internship.drop('Unnamed: 0',axis=1,inplace = True)
student.drop('Unnamed: 0',axis=1,inplace = True)
student_exp.drop('Unnamed: 0',axis=1,inplace = True)

In [14]:
#data['Preferred_location'] = data['Preferred_location'].apply(lambda x : 'NS' if pd.isnull(x) else x)
#internship['Skills_required'] = internship['Skills_required'].apply(lambda x : 'NS' if pd.isnull(x) else x)
#internship['skill_compress_str'] = internship['skill_compress_str'].apply(lambda x : 'NS' if pd.isnull(x) else x)
#datai = pd.merge(data,internship,how = 'left', on = 'Internship_ID')
#datais = pd.merge(datai,student,how = 'left', on = 'Student_ID')
#datais['Stipend1']= datais['Stipend1'].fillna(0)
#datais_dates = datais[['Student_ID','Earliest_Start_Date']]
#student_exp['Experience_Type'] = student_exp['Experience_Type'].fillna('NE')
#student_exp['Profile'] = student_exp['Profile'].fillna('NS')
#student_exp['exp_days'] = 0
#student_exp = student_exp[student_exp['Start.Date'].notnull()]
#student_exp['Student_ID'] = str(student_exp['Student_ID'])
student_exp['Profile'] = str(student_exp['Profile'])
#student_exp['Profile'] = student_exp[['Student_ID','Profile']].groupby(['Student_ID'])['Profile'].transform(lambda x: ','.join(x))
#student_exp['Experience_Type'] = student_exp[['Student_ID','Experience_Type']].groupby(['Student_ID'])['Experience_Type'].transform(lambda x: ','.join(x))
#student_exp_unique = student_exp[['Student_ID','Experience_Type','Profile']].drop_duplicates()
#dataise = pd.merge(datais,student_exp_unique,on = 'Student_ID')
#student_exp.dtypes

MemoryError: 

In [5]:
dataise['avg_stipend'] = 0
dataise['avg_stipend'] = np.where(dataise['Stipend2'].isnull(),dataise['Stipend1'],(dataise['Stipend1']+dataise['Stipend2'])/2)
dataise.drop('Stipend2',axis=1,inplace=True)
dataise['percentage_pg'] = np.where(dataise['PG_scale'] == 4,(dataise['Performance_PG']+1)*20,np.where(dataise['PG_scale'] ==10 ,
                                                                                                   dataise['Performance_PG']*10,
                                                                                                   dataise['Performance_PG']))
dataise['percentage_ug'] = np.where(dataise['UG_Scale'] == 4,(dataise['Performance_UG']+1)*20,np.where(dataise['UG_Scale'] ==10 ,
                                                                                                   dataise['Performance_UG']*10,
                                                                                                   dataise['Performance_UG']))
dataise['student_stipend'] = np.where(dataise['Expected_Stipend']=='No Expectations',0 ,np.where(dataise['Expected_Stipend']==
                                                                                     '2-5K',2000,np.where(dataise['Expected_Stipend']==
                                                                                                         '5-10K',5000,10000)))
dataise['Stipend_low_match'] = np.where(dataise['Stipend1'] >= dataise['student_stipend'],'Yes','No')
dataise['Stipend_avg_match'] = np.where(dataise['avg_stipend'] >= dataise['student_stipend'],'Yes','No')
dataise.drop('Performance_PG',axis=1,inplace = True)
dataise.drop('Performance_UG',axis=1,inplace = True)
dataise['location_match'] = np.where(dataise['Preferred_location']=='NS','Yes',np.where(dataise['Preferred_location']==
                                                                                     dataise['Internship_Location'],'Yes','No'))


In [6]:
dataise['Internship_Profile'] = [nltk.word_tokenize(sent) for sent in dataise['Internship_Profile']]
dataise['Skills_required'] = [nltk.word_tokenize(sent) for sent in dataise['Skills_required']]
dataise['skill_compress_str'] = [nltk.word_tokenize(sent) for sent in dataise['skill_compress_str']]
dataise['Profile'] = [nltk.word_tokenize(sent.decode('utf8', 'ignore')) for sent in dataise['Profile']]

In [ ]:
dataise['degree_base'] = 'empty'
def
for i in range(len(dataise)):
    dataise['Degree'][i] = str(dataise['Degree'][i])
    dataise['degree_base'][i] = np.where(re.search(r'^(u|b|B|U)',dataise['Degree'][i]),'UG',np.where(
            re.search(r'^(m|p|M|P)',dataise['Degree'][i]),'PG','Others'))

In [7]:
def common_elements(list1, list2):
    a =  [element for element in list1 if element in list2]
    return len(a)
dataise['profile_common'] = 0
dataise['skill_common'] = 0
dataise['skill_common2'] = 0
for i in range(len(dataise)):
    dataise['profile_common'][i] = common_elements(dataise['Internship_Profile'][i],dataise['Profile'][i])
    dataise['skill_common'][i] = common_elements(dataise['Skills_required'][i],dataise['Profile'][i])
    dataise['skill_common2'][i] = common_elements(dataise['skill_compress_str'][i],dataise['Profile'][i])

C:\Users\user\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda2\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda2\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


KeyboardInterrupt: 

In [ ]:
data_new = dataise[x for x in dataise.columns ]
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
var_to_encod = ['Expected_Stipend','Preferred_location','Is_Part_Time','Internship_Profile','Internship_Type','Internship_Location','Internship_category',
                 'Stipend_Type','skill_compress_str','Institute_Category','Degree','Stream','Year_of_graduation',
                 'PG_scale','UG_Scale','Experience_Type','location_match','Stipend_low_match','Stipend_avg_match','Current_year','degree_base']
for col in var_to_encode:
    data_new[col] = le.fit_transform(data_new[col])
data_new = pd.get_dummies(data_new, columns=var_to_encode)

In [ ]:
train = data_new.loc[data_new['source'] == 'train']
test = data_new.loc[data_new['source']=='test']
train.drop('source',axis=1,inplace=True)
test.drop('Is_Shortlisted',axis=1,inplace=True)
predictors = [x for x in train.columns if x not in ['Internship_ID','Student_ID','Is_Shortlisted']]
gbm = GradientBoostingClassifier(random_state=143)
modelfit(gbm,train,test,predictors)

In [ ]:
test['Is_Shortlisted'] = gbm.predict(test[predictors])
result = test[['Internship_ID','Student_ID','Is_Shortlisted']]
result.to_csv("result.csv")


Unnamed: 0              int64
Student_ID              int64
Institute_Category     object
Degree                 object
Stream                 object
Current_year           object
Year_of_graduation      int64
Performance_PG        float64
PG_scale                int64
Performance_UG        float64
UG_Scale                int64
Performance_12th      float64
Performance_10th      float64
Freq                    int64
dtype: object